In [1]:
from eolearn.core.eoworkflow import LinearWorkflow, Dependency
from eolearn.core.eodata import FeatureType

from sentinelhub import BBox, CRS

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import datetime

from eolearn.core import SaveToDisk, LoadFromDisk
from eolearn.io import S2L1CWCSInput, AddSen2CorClassificationFeature, DEMWCSInput, S2L2AWCSInput, L8L1CWCSInput, ExportToTiff

In [ ]:
# # reference time

# date_first_start = '2019-04-18'
# date_first_end = '2019-04-20'

# date_last_start = '2019-08-21'
# date_last_end = '2019-08-22'

In [28]:
roi_bbox = BBox(bbox=[27.11, 51.88, 27.57, 51.64], crs=CRS.WGS84)
time_interval = ('2019-04-18', '2019-04-20')

In [29]:
layer = 'BANDS-S2-L1C'

input_task = S2L1CWCSInput(layer=layer,
                           resx='20m', resy='20m',
                           maxcc=.8, time_difference=datetime.timedelta(hours=2))

add_ndvi = S2L1CWCSInput(layer='NDVI')

tiff_location = './predicted_tiff_D'

add_dem = DEMWCSInput(layer='DEM')

#add_l2a = S2L2AWCSInput(layer='BANDS-S2-L2A')

add_sen2cor = AddSen2CorClassificationFeature('SCL', layer='BANDS-S2-L2A')

save = SaveToDisk('io_example', overwrite_permission=2, compress_level=1)

In [ ]:
workflow = LinearWorkflow(input_task, add_ndvi, add_sen2cor, add_dem, save)

result = workflow.execute({input_task: {'bbox': roi_bbox, 'time_interval': time_interval},
            save: {'eopatch_folder': 'eopatch'}})

In [26]:
eopatch = result[save]
eopatch

EOPatch(
  data: {
    BANDS-S2-L1C: numpy.ndarray(shape=(1, 1335, 1583, 13), dtype=float32)
    NDVI: numpy.ndarray(shape=(1, 1335, 1583, 1), dtype=float32)
  }
  mask: {
    IS_DATA: numpy.ndarray(shape=(1, 1335, 1583, 1), dtype=bool)
    SCL: numpy.ndarray(shape=(1, 1335, 1583, 1), dtype=int32)
  }
  scalar: {}
  label: {}
  vector: {}
  data_timeless: {
    DEM: numpy.ndarray(shape=(1335, 1583, 1), dtype=float32)
  }
  mask_timeless: {}
  scalar_timeless: {}
  label_timeless: {}
  vector_timeless: {}
  meta_info: {
    maxcc: 0.8
    service_type: 'wcs'
    size_x: '20m'
    size_y: '20m'
    time_difference: datetime.timedelta(seconds=7200)
    time_interval: ('2019-04-18', '2019-04-20')
  }
  bbox: BBox(((27.11, 51.64), (27.57, 51.88)), crs=EPSG:4326)
  timestamp: [datetime.datetime(2019, 4, 19, 9, 34, 38)]
)

In [27]:
plt.figure(figsize=(10,10))
plt.imshow(eopatch.data['BANDS-S2-L1C'][3][..., [3,2,1]] * 2.5, vmin=0, vmax=1);

IndexError: index 3 is out of bounds for axis 0 with size 1

<Figure size 360x360 with 0 Axes>

In [17]:
plt.figure(figsize=(10,10))
plt.imshow(eopatch.data['NDVI'][3].squeeze());

IndexError: index 3 is out of bounds for axis 0 with size 1

<Figure size 720x720 with 0 Axes>

In [8]:
plt.figure(figsize=(10,10))
plt.imshow(eopatch.data['BANDS-S2-L2A'][3][...,[3,2,1]] * 2.5, vmin=0, vmax=1);

KeyError: 'BANDS-S2-L2A'

<Figure size 720x720 with 0 Axes>